# HW #4: Data Analysis of Building Energy Benchmarking Data


This project focuses on analyzing the City of Calgary’s Building Energy Benchmarking dataset as part of a data science assignment. The objective is to explore, preprocess, and visualize the dataset using a variety of Python tools and techniques. The analysis will involve data cleaning, the application of Regular Expressions (Regex) for text standardization, and the use of Pandas and NumPy for data manipulation and aggregation. Additionally, visualizations will be generated using Matplotlib to identify trends and insights. This process will help to better understand the relationships between building characteristics and energy performance in Calgary.

# Part 1: Data Cleaning and Preprocessing

## 1.1 Load and Inspect the Dataset
__1.1.1__ Load the dataset and display its shape, column names, and data types.

__1.1.2__  Identify and list the number of missing values in each column..

In [217]:
import pandas as pd

# 1.1.1 Load the dataset and display its shape, column names, and data types.
df = pd.read_csv("Building_Energy_Benchmarking.csv")
df.shape             #shape
df.columns           #Columns names
df.info              # data types 


# 1.1.2. Identify and list the number of missing values in each column..
number_missing_values = df.isnull().sum()    #Missing data for each column


########################## Code to display the output in a well-organized format #####################################
# 1.1.1 Load the dataset and display its shape, column names, and data types.
print('\n')
print((' '*10),'1.1.1. LOAD THE DATASET AND DISPLAY ITS SHAPE, COLUMNS, AND DATA TYPES.', '\n')
print(('_'*40), 'SHAPE OF THE df',('_'*40 ))
print('\n','The shape of the df is',df.shape[0],'rows and',df.shape[1],'columns','\n')
print(('_'*38), 'NAME OF EACH COLUMN',('_'*38 ), '\n')
print(df.columns)
print(('_'*36), 'DATA TYPE IN EACH COLUMN',('_'*36 ))
print(df.info(), '\n')
print('\n')

# 1.1.2. Identify and list the number of missing values in each column.
print((' '*13),'1.1.2. IDENTIFY AND LIST THE NUMBER OF MISSING VALUES IN EACH COLUMN')
print('_'*97)
print(number_missing_values)



           1.1.1. LOAD THE DATASET AND DISPLAY ITS SHAPE, COLUMNS, AND DATA TYPES. 

________________________________________ SHAPE OF THE df ________________________________________

 The shape of the df is 494 rows and 31 columns 

______________________________________ NAME OF EACH COLUMN ______________________________________ 

Index(['Property Id', 'Property Name', 'Address 1', 'City', 'Postal Code',
       'Province', 'Primary Property Type - Self Selected',
       'Number of Buildings', 'Year Built',
       'Property GFA - Self-Reported (m²)', 'ENERGY STAR Score',
       'Site Energy Use (GJ)', 'Weather Normalized Site Energy Use (GJ)',
       'Site EUI (GJ/m²)', 'Weather Normalized Site EUI (GJ/m²)',
       'Source Energy Use (GJ)', 'Weather Normalized Source Energy Use (GJ)',
       'Source EUI (GJ/m²)', 'Weather Normalized Source EUI (GJ/m²)',
       'Total GHG Emissions (Metric Tons CO2e)',
       'Total GHG Emissions Intensity (kgCO2e/m²)',
       'Direct GHG Emissions (M

## 1.2 Handling Missing Data
__1.2.1__ Drop columns with more than 40% missing values.


__1.2.2__ For numerical columns, fill missing values with the median of their respective column.

__1.2.3__ For categorical columns, fill missing values with the mode of their respective column.

In [219]:
import pandas as pd 
from sklearn.impute import SimpleImputer

# 1.2.1 Drop columns with more than 40% missing values.
percent_missing = df.isnull().mean() * 100                          #Missing data for each column
drop_columns = df.columns[(percent_missing/100) > 0.4].tolist()     #Identify columns or rows with more than 40% missing values
dfc = df.drop(drop_columns, axis=1)                                 #Drop columns in a new DF called dfc 



# 1.2.2 For numerical columns, fill missing values with the median of their respective column.
num_cols = dfc.select_dtypes(include=['float64', 'int64']).columns  #Identify numerical columns
median_imputer = SimpleImputer(strategy = 'median')                 #Establish with what to replace missing values
dfc[num_cols] = median_imputer.fit_transform(dfc[num_cols])         #Replace missing values with median
print(num_cols.isnull().mean() * 100, '\n')



# 1.2.3 For categorical columns, fill missing values with the mode of their respective column.
categor_cols = dfc.columns.difference(num_cols)                                  #Identify categorical columns != numerical columns
dfc[categor_cols] = dfc[categor_cols].apply(lambda x: x.fillna(x.mode()[0]))     #Replace with mode in the missing values 



########################## Code to display the output in a well-organized format #####################################
# 1.2.1 Drop columns with more than 40% missing values.
print((' '*27),'1.1.2. DROP COLUMNS WITH MORE THAN 40% MISSING VALUES', '\n')
print(('_'*26), 'PERCENTAGE OF MISSING DATA FOR EACH COLUMN',('_'*26))
print(percent_missing, '\n')
print(('_'*25), 'COLUMNS DROPED WITH MORE THAN 40% MISSING VALUES',('_'*25))
print(drop_columns)
print('_'*100, '\n')

# 1.2.2 For numerical columns, fill missing values with the median of their respective column.
print('\n',(' '*6), '1.2.2 FOR NUMERICAL COLUMNS, FILL MISSING VALUES WITH THE MEDIAN OF THEIR RESPECTIVE COLUMN', '\n')
print(('_'*15), 'PERCENTAGE OF MISSING DATA IN EACH NUMERICAL COLUMN AFTER IMPUTATION',('_'*15))
print(dfc[num_cols].isnull().mean() * 100)
print('_'*100, '\n')

# 1.2.3 For categorical columns, fill missing values with the mode of their respective column.
print((' '*6), ' 1.2.3 FOR CATEGORICAL COLUMNS, FILL MISSING VALUES WITH THE MODE OF THEIR RESPECTIVE COLUMN', '\n')
print(('_'*14), 'PERCENTAGE OF MISSING DATA IN EACH CATEGORICAL COLUMN AFTER IMPUTATION',('_'*14))
print(dfc[categor_cols].isnull().mean() * 100)
print('_'*100, '\n')

0.0 

                            1.1.2. DROP COLUMNS WITH MORE THAN 40% MISSING VALUES 

__________________________ PERCENTAGE OF MISSING DATA FOR EACH COLUMN __________________________
Property Id                                                               0.000000
Property Name                                                             0.000000
Address 1                                                                 0.000000
City                                                                      0.000000
Postal Code                                                               0.000000
Province                                                                  0.000000
Primary Property Type - Self Selected                                     0.000000
Number of Buildings                                                       0.000000
Year Built                                                                0.000000
Property GFA - Self-Reported (m²)                                 

## 1.3 Extracting and Cleaning Data Using Regex
• **Use Regex only to:**

__1.3.1__ Extract numeric values from text-based numeric columns (e.g., Property GFA,
Energy Use, Emissions).

__1.3.1__ Standardize Postal Codes to follow the Canadian format (A1A 1A1).

__1.3.1__ Clean and extract meaningful text from Property Names and Addresses.

__1.3.1__ Ensure extracted values are properly converted to numerical types for analysis.

In [306]:
import pandas as pd
import re

# 1.3.1 Extract numeric values from text-based numeric columns (e.g., Property GFA, Energy Use, Emissions).
def numeric_object(df):                                             #Function name
    numeric_columns = []                                            #columns names in a list                                    
    for col in df.select_dtypes(include=['object']).columns:
        if df[col].astype(str).str.match(r'-?\d+\.?\d*', na=False).all():  # Verifica si hay al menos un número válido
            numeric_columns.append(col)
    return numeric_columns

list_columns = numeric_object(dfc)    #Llamar a la funcion 

for col in list_columns[1:]:          #Codigo para cambiar str a float
    if col in dfc.columns:            # Extraer lista de números encontrados
        dfc[col] = (dfc[col].apply(lambda str: re.findall(r'-?\d+\.?\d*', str)))            

#dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 26 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Property Id                                 494 non-null    float64
 1   Property Name                               494 non-null    object 
 2   Address 1                                   494 non-null    object 
 3   City                                        494 non-null    object 
 4   Postal Code                                 491 non-null    object 
 5   Province                                    494 non-null    object 
 6   Primary Property Type - Self Selected       494 non-null    object 
 7   Number of Buildings                         494 non-null    float64
 8   Year Built                                  494 non-null    float64
 9   Property GFA - Self-Reported (m²)           494 non-null    float64
 10  Site Energy Us

In [288]:
import pandas as pd
import re

# 1.3.1 Standardize Postal Codes to follow the Canadian format (A1A 1A1).
def Canadian_format(postal):
    match = re.match(r'([a-zA-Z]\d[a-zA-Z])\s?(\d[a-zA-Z]\d)', str(postal).upper().strip())  
    return f"{match.group(1)} {match.group(2)}" if match else None  

dfc["Postal Code"] = dfc["Postal Code"].apply(Canadian_format)

print(dfc["Postal Code"].head())

0    T2H 0Z4
1    T2E 7L7
2    T2G 0W2
3    T2G 0G2
4    T2G 4M7
Name: Postal Code, dtype: object


In [302]:
import pandas as pd
import re
df = dfc.copy()

df['Property Name'] = df['Property Name'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x).strip())
df['Address 1'] = df['Address 1'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s,]', '', x).strip())
df[['Property Name', 'Address 1']].head(20)


,Property Name,Address 1
0,Acadia Aquatic Fitness Centre,9009 Fairmount Dr SE
1,Ad Valorem,2924 11 ST NE
2,Alberta Trade Centre,315 10 AV SE
3,Andrew Davison,133 6 AV SE
4,Animal Services Centre,2201 PORTLAND ST SE
5,Apparatus Repair Shop and Spare Apparatus Shop,1725 18 AV NE
6,Beltline Aquatic Fitness Centre,221 12 Av SW
7,Bob Bohan Aquatic and Fitness Centre,4812 14 Av SE
8,Bowmont Civic Building,5000 Bowness Civic Building
9,Calgary Public Building,205 8 AV SE


In [296]:
dfc[['Property Name', 'Address 1']].head(20)

,Property Name,Address 1
0,Acadia Aquatic & Fitness Centre,9009 Fairmount Dr SE
1,Ad Valorem,2924 11 ST NE
2,Alberta Trade Centre,315 10 AV SE
3,Andrew Davison,133 6 AV SE
4,Animal Services Centre,2201 PORTLAND ST SE
5,Apparatus Repair Shop and Spare Apparatus Shop,1725 18 AV NE
6,Beltline Aquatic & Fitness Centre,221 12 Av SW
7,Bob Bohan Aquatic and Fitness Centre,4812 14 Av SE
8,Bowmont Civic Building,5000 Bowness Civic Building
9,Calgary Public Building,205 8 AV SE
